In [3]:
SQL_SOURCE = 'https://yc3300.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

## Task 1 — Familiarize with SQL Clauses
• Sort data by start_station_id, tripduration                 
- Only checking trips with duration <= 3 hours
• Only show the top/last 10 records (aka head and tail in SQL)               
• List all unique start_station_id values         
• Aggregation functions:                
- Count the number of trips (aka wc -l in SQL)
- Find the average/min/max trip duration

In [52]:
task1 = '''
SELECT DISTINCT start_station_id, 
COUNT(start_station_id) AS trip_counts, 
AVG(tripduration) AS ave_tripduration,
MIN(tripduration) AS min_tripduration,
MAX(tripduration) AS max_tripduration
FROM citibike
WHERE tripduration<=10800
GROUP BY start_station_id
ORDER BY start_station_id DESC, ave_tripduration DESC
LIMIT 10
'''

In [53]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task1)), sep=',')

,start_station_id,trip_counts,ave_tripduration,min_tripduration,max_tripduration
0,3002,184,714.646739,79,7225
1,2023,91,771.978022,108,2454
2,2022,96,979.062500,107,2495
3,2021,141,738.815603,90,6592
4,2017,86,763.383721,109,2476
5,2012,256,714.519531,79,2026
6,2010,76,763.184211,167,2610
7,2009,107,777.794393,60,2765
8,2008,86,832.220930,94,3476
9,2006,79,1130.531646,90,3459


## Task 2 — Working with date/time
• Selecting trips started on Feb-02-2015 only                                    
• Selecting trips started on the weekends 
    -What are average trip duration during weekends?            
• Can we do the same for weekday?

In [70]:
task2_1 = '''
SELECT gender, birth_year, end_station_id, start_station_id,tripduration,usertype, stoptime, starttime FROM citibike 
WHERE starttime >= '2015-02-02 00:00'
AND starttime < '2015-02-03 00:00'
ORDER BY starttime DESC
LIMIT 10
'''

In [71]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_1)), sep=',')

,gender,birth_year,end_station_id,start_station_id,tripduration,usertype,stoptime,starttime
0,2,1976,302,285,733,Subscriber,2015-02-03 00:10:00+00,2015-02-02 23:58:00+00
1,1,1961,537,453,634,Subscriber,2015-02-03 00:05:00+00,2015-02-02 23:55:00+00
2,1,1988,394,345,689,Subscriber,2015-02-03 00:02:00+00,2015-02-02 23:50:00+00
3,1,1988,417,224,188,Subscriber,2015-02-02 23:50:00+00,2015-02-02 23:47:00+00
4,1,1977,440,484,371,Subscriber,2015-02-02 23:52:00+00,2015-02-02 23:45:00+00
5,1,1979,397,496,2111,Subscriber,2015-02-03 00:18:00+00,2015-02-02 23:43:00+00
6,1,1983,439,280,518,Subscriber,2015-02-02 23:52:00+00,2015-02-02 23:43:00+00
7,2,1982,489,505,813,Subscriber,2015-02-02 23:56:00+00,2015-02-02 23:42:00+00
8,1,1987,507,509,764,Subscriber,2015-02-02 23:55:00+00,2015-02-02 23:42:00+00
9,1,1986,368,402,560,Subscriber,2015-02-02 23:46:00+00,2015-02-02 23:37:00+00


In [98]:
task2_2 = '''
SELECT EXTRACT(DOW FROM starttime) AS date_of_week,
AVG(tripduration) AS ave_tripduration 
FROM citibike 
WHERE extract(DOW FROM starttime) IN (0,6)
GROUP BY date_of_week
ORDER BY ave_tripduration DESC
LIMIT 10
'''

In [99]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_2)), sep=',')

,date_of_week,ave_tripduration
0,6,686.460825
1,0,638.149200


In [102]:
task2_3 = '''
SELECT EXTRACT(DOW FROM starttime) AS date_of_week,
AVG(tripduration) AS ave_tripduration 
FROM citibike 
WHERE extract(DOW FROM starttime) NOT IN (0,6)
GROUP BY date_of_week
ORDER BY ave_tripduration DESC
LIMIT 10
'''

In [103]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_3)), sep=',')

,date_of_week,ave_tripduration
0,1,920.862234
1,2,767.224443
2,3,697.556559
3,5,637.116968
4,4,623.396720


## Task 3 — Working with Space
• Showing the list of start station locations 
- Using GROUP BY
• Showing the number of trips started per station        
• … but only for stations within 500m of Time Square!
- The coordinates of Time Square is (40.7577,-73.9857)

In [148]:
task3_1 = '''
SELECT start_station_latitude, start_station_longitude,
CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude,start_station_longitude)) as the_geom_webmercator,
MIN(cartodb_id) as cartodb_id
FROM citibike
GROUP BY start_station_latitude, start_station_longitude
'''

In [149]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task3_1)), sep=',') #showig the list of start_station using GROUPBY

,start_station_latitude,start_station_longitude,the_geom_webmercator,cartodb_id
0,40.686919,-73.976682,0101000020110F0000863F88A4096A5FC19D1DAFD5DFF1...,84
1,40.753202,-73.977987,0101000020110F0000BB2EC9F82D6A5FC19150E1B461FB...,412
2,40.715422,-74.011220,0101000020110F0000D62721D3CA6D5FC157A5D529F6F5...,516
3,40.745497,-74.001971,0101000020110F000060197671C96C5FC197A854AF46FA...,359
4,40.744751,-73.999154,0101000020110F0000993A68067B6C5FC1B222DA492BFA...,78
5,40.742065,-74.004432,0101000020110F00004FF9F3E90D6D5FC1F284479FC8F9...,1019
6,40.740964,-73.986022,0101000020110F00003816CC930D6B5FC17B393D28A0F9...,110
7,40.763406,-73.977225,0101000020110F00006B6C9CBF186A5FC109C57CA0D8FC...,7
8,40.716059,-73.991908,0101000020110F00008B8D725EB16B5FC1BF41718A0DF6...,460
9,40.714948,-74.002345,0101000020110F0000BA97F1D5D36C5FC13D326BC3E4F5...,799


In [152]:
task3_2 = '''
SELECT
 start_station_id,
 start_station_latitude,
 start_station_longitude,
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id,
 COUNT(tripduration) as numtrips
FROM citibike
GROUP BY
 start_station_id,
 start_station_latitude,
 start_station_longitude
 '''

In [153]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task3_2)), sep=',')

,start_station_id,start_station_latitude,start_station_longitude,the_geom_webmercator,cartodb_id,numtrips
0,438,40.727791,-73.985649,0101000020110F0000B77CA834036B5FC19EF5005EBCF7...,79,156
1,342,40.717400,-73.980166,0101000020110F0000BEA7E6966A6A5FC1E90E1AC83EF6...,609,78
2,242,40.697883,-73.973503,0101000020110F0000E097312EB1695FC111C4474A72F3...,454,46
3,161,40.729170,-73.998102,0101000020110F00009B8267C45D6C5FC1577C6302EFF7...,185,203
4,373,40.693317,-73.953820,0101000020110F0000C12D0F658D675FC1323CC0ADCAF2...,6,27
5,168,40.739713,-73.994564,0101000020110F000071CD414CFB6B5FC149C2933772F9...,49,327
6,502,40.714215,-73.981346,0101000020110F00002877F4708B6A5FC1ED3B00D9C9F5...,289,151
7,421,40.695734,-73.971297,0101000020110F00000C641AC573695FC10D6A916323F3...,238,37
8,511,40.729387,-73.977724,0101000020110F000049C345A6266A5FC17E86BFF6F6F7...,69,279
9,422,40.770513,-73.988038,0101000020110F0000CECBC1AD456B5FC149916BC5DDFD...,1386,100


In [154]:
task3_3 = '''SELECT
 start_station_id,
 start_station_latitude,
 start_station_longitude,
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id,
 COUNT(tripduration) as numtrips
FROM citibike
WHERE
ST_DWithin(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )::geography,
 CDB_LatLng(40.7577,-73.9857)::geography,
 500)
GROUP BY
 start_station_id,
 start_station_latitude,
 start_station_longitude
 '''

In [155]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task3_3)), sep=',') #times square 500m

,start_station_id,start_station_latitude,start_station_longitude,the_geom_webmercator,cartodb_id,numtrips
0,477,40.756405,-73.990026,0101000020110F0000F55695027D6B5FC186E5BB69D7FB...,33,507
1,2021,40.759291,-73.988597,0101000020110F00001607D538556B5FC119A6CA6F41FC...,124,141
2,465,40.755136,-73.986580,0101000020110F000004AF971C1D6B5FC17A48F3C1A8FB...,54,251
3,493,40.756800,-73.982912,0101000020110F00009BB87D02B76A5FC146C144E9E5FB...,19,141
4,524,40.755273,-73.983169,0101000020110F0000F22A632FBE6A5FC1363A28CFADFB...,1115,112
5,529,40.757570,-73.990985,0101000020110F00003F9300B2976B5FC1B8B88F3102FC...,783,221
6,173,40.760647,-73.984427,0101000020110F000020D6742CE16A5FC112F4D73D73FC...,42,213


## Task 4 — Putting it all together
• Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare!  
• Extra: create lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours. The number of trips per each pair of stations are output as attributes of these lines

In [1]:
task4= '''
SELECT
 start_station_id,
 start_station_latitude,
 start_station_longitude,
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude)
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id,
 AVG(tripduration) as avg_duration
FROM citibike
WHERE
ST_DWithin(
 CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )::geography,
 CDB_LatLng(40.7577,-73.9857)::geography,
 500)
AND
extract(DOW FROM starttime) IN (0,6)

GROUP BY
 start_station_id,
 start_station_latitude,
 start_station_longitude
ORDER BY avg_duration DESC
LIMIT 1
'''

In [5]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task4)), sep=',') 

,start_station_id,start_station_latitude,start_station_longitude,the_geom_webmercator,cartodb_id,avg_duration
0,173,40.760647,-73.984427,0101000020110F000020D6742CE16A5FC112F4D73D73FC...,42,1010.104167


In [10]:
task4_extra = '''
SELECT 
start_station_latitude, start_station_longitude,end_station_latitude,end_station_longitude,
st_makeline(CDB_transformToWebmercator(CDB_latlng(start_station_latitude,start_station_longitude)), 
                   CDB_transformToWebmercator(CDB_latlng(end_station_latitude,end_station_longitude)))as the_geom_webmercator,Min(cartodb_id) as cartodb_id,count(tripduration) as numtrips FROM citibike
where st_dwithin(cdb_latlng(start_station_latitude,start_station_longitude)::geography, cdb_latlng(40.7577,-73.9857)::geography,500) and tripduration<7200
 
group by start_station_latitude, start_station_longitude,end_station_latitude,end_station_longitude
'''

In [11]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task4_extra)), sep=',') 

,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,the_geom_webmercator,cartodb_id,numtrips
0,40.755273,-73.983169,40.739126,-73.979738,0102000020110F000002000000F22A632FBE6A5FC1363A...,12043,1
1,40.756800,-73.982912,40.743943,-73.979661,0102000020110F0000020000009BB87D02B76A5FC146C1...,34407,1
2,40.755136,-73.986580,40.750664,-74.001768,0102000020110F00000200000004AF971C1D6B5FC17A48...,30188,1
3,40.757570,-73.990985,40.757570,-73.990985,0102000020110F0000020000003F9300B2976B5FC1B8B8...,7566,5
4,40.760647,-73.984427,40.746920,-74.004519,0102000020110F00000200000020D6742CE16A5FC112F4...,8850,4
5,40.757570,-73.990985,40.739445,-73.976806,0102000020110F0000020000003F9300B2976B5FC1B8B8...,31004,1
6,40.760647,-73.984427,40.743453,-74.000040,0102000020110F00000200000020D6742CE16A5FC112F4...,4636,2
7,40.756405,-73.990026,40.761628,-73.972924,0102000020110F000002000000F55695027D6B5FC186E5...,6788,6
8,40.755273,-73.983169,40.762272,-73.987882,0102000020110F000002000000F22A632FBE6A5FC1363A...,20181,2
9,40.759291,-73.988597,40.747659,-73.984907,0102000020110F0000020000001607D538556B5FC119A6...,26631,1
